In [1]:
!pip install nptdms

In [2]:
from nptdms import TdmsFile
from google.cloud import storage

In [3]:
from google.cloud import storage

client = storage.Client()
bucket = client.bucket("emotor-dataset-raw")

print([b.name for b in bucket.list_blobs(prefix="current,temp")])

['current,temp/0Nm_BPFI_03.tdms', 'current,temp/0Nm_BPFI_10.tdms', 'current,temp/0Nm_BPFI_30.tdms', 'current,temp/0Nm_BPFO_03.tdms', 'current,temp/0Nm_BPFO_10.tdms', 'current,temp/0Nm_BPFO_30.tdms', 'current,temp/0Nm_Misalign_01.tdms', 'current,temp/0Nm_Misalign_03.tdms', 'current,temp/0Nm_Misalign_05.tdms', 'current,temp/0Nm_Normal.tdms', 'current,temp/0Nm_Unbalance_0583mg.tdms', 'current,temp/0Nm_Unbalance_1169mg.tdms', 'current,temp/0Nm_Unbalance_1751mg.tdms', 'current,temp/0Nm_Unbalance_2239mg.tdms', 'current,temp/0Nm_Unbalance_3318mg.tdms', 'current,temp/2Nm_BPFI_03.tdms', 'current,temp/2Nm_BPFI_10.tdms', 'current,temp/2Nm_BPFI_30.tdms', 'current,temp/2Nm_BPFO_03.tdms', 'current,temp/2Nm_BPFO_10.tdms', 'current,temp/2Nm_BPFO_30.tdms', 'current,temp/2Nm_Misalign_01.tdms', 'current,temp/2Nm_Misalign_03.tdms', 'current,temp/2Nm_Misalign_05.tdms', 'current,temp/2Nm_Normal.tdms', 'current,temp/2Nm_Unbalance_0583mg.tdms', 'current,temp/2Nm_Unbalance_1169mg.tdms', 'current,temp/2Nm_Unbal

In [4]:
from google.cloud import storage
from nptdms import TdmsFile

# =============================
# 1. CONFIG
# =============================
bucket_name = "emotor-dataset-raw"
file_path = "current,temp/0Nm_BPFI_03.tdms"
local_file = "0Nm_BPFI_03.tdms"

# =============================
# 2. GCP CLIENT
# =============================
client = storage.Client()

# =============================
# 3. DOWNLOAD FILE
# =============================
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)

blob.download_to_filename(local_file)
print("Descargado:", local_file)

# =============================
# 4. REED TDMS
# =============================
tdms_file = TdmsFile.read(local_file)
df = tdms_file.as_dataframe()

print("Shape:", df.shape)
print(df.head())


Descargado: 0Nm_BPFI_03.tdms
Shape: (1536492, 5)
   /'Log'/'cDAQ9185-1F486B5Mod1/ai0'  /'Log'/'cDAQ9185-1F486B5Mod1/ai1'  \
0                          27.607992                          28.217591   
1                          27.607992                          28.217591   
2                          27.607992                          28.217591   
3                          27.607992                          28.217591   
4                          27.607992                          28.217591   

   /'Log'/'cDAQ9185-1F486B5Mod2/ai0'  /'Log'/'cDAQ9185-1F486B5Mod2/ai2'  \
0                           1.894377                           0.949463   
1                           2.128889                           0.926111   
2                           2.373001                           0.882154   
3                           2.087747                           1.193974   
4                           2.393572                           1.125291   

   /'Log'/'cDAQ9185-1F486B5Mod2/ai3'  
0         

In [5]:
!pip install nptdms pandas pyarrow google-cloud-storage tqdm


In [6]:
from google.cloud import storage

BUCKET_NAME = "emotor-dataset-raw"

def test_connection():
    try:
        client = storage.Client()
        print("✅ Cliente GCP creado correctamente")

        # Listar los primeros 10 blobs del bucket
        blobs = client.list_blobs(BUCKET_NAME)

        print(f"📂 Contenido del bucket '{BUCKET_NAME}':")
        for i, blob in enumerate(blobs):
            print(" -", blob.name)
            if i >= 10:
                break

        print("\n🎉 Conexión exitosa. Puedes leer el bucket sin problemas.")

    except Exception as e:
        print("❌ Error al conectar o listar objetos:")
        print(e)

if __name__ == "__main__":
    test_connection()


✅ Cliente GCP creado correctamente
📂 Contenido del bucket 'emotor-dataset-raw':
 - current,temp/0Nm_BPFI_03.tdms
 - current,temp/0Nm_BPFI_10.tdms
 - current,temp/0Nm_BPFI_30.tdms
 - current,temp/0Nm_BPFO_03.tdms
 - current,temp/0Nm_BPFO_10.tdms
 - current,temp/0Nm_BPFO_30.tdms
 - current,temp/0Nm_Misalign_01.tdms
 - current,temp/0Nm_Misalign_03.tdms
 - current,temp/0Nm_Misalign_05.tdms
 - current,temp/0Nm_Normal.tdms
 - current,temp/0Nm_Unbalance_0583mg.tdms

🎉 Conexión exitosa. Puedes leer el bucket sin problemas.


In [ ]:
import io
import numpy as np
import pandas as pd
from tqdm import tqdm
from nptdms import TdmsFile
from google.cloud import storage

BUCKET_NAME = "emotor-dataset-raw"
INPUT_PREFIX = "current,temp/"
OUTPUT_PREFIX = "current_temp_short/"

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)


def list_files(prefix):
    """List all files under a GCP prefix."""
    return [
        blob.name for blob in client.list_blobs(BUCKET_NAME, prefix=prefix)
        if blob.name.lower().endswith(".tdms")
    ]


def convert_tdms_to_parquet(blob_name):
    print(f"\n🔄 Processing: {blob_name}")

    # Download file
    blob = bucket.blob(blob_name)
    tdms_bytes = blob.download_as_bytes()

    # Read TDMS
    tdms_file = TdmsFile(io.BytesIO(tdms_bytes))

    dfs = []

    for group in tdms_file.groups():
        for channel in group.channels():

            # Convert data first
            data_array = np.array(channel[:])

            # Skip non-numeric channels
            if not np.issubdtype(data_array.dtype, np.number):
                print(f"⚠️ Skipped non-numeric channel: {channel.name} ({data_array.dtype})")
                continue

            # Time track
            try:
                time_data = np.array(channel.time_track())
            except Exception:
                time_data = np.arange(len(channel))

            df = pd.DataFrame({
                "time": time_data,
                channel.name: data_array
            })

            dfs.append(df)

    if len(dfs) == 0:
        print("❌ No numeric channels found. Skipping file.")
        return

    # Merge all channels on time
    df_final = dfs[0]
    for df_other in dfs[1:]:
        df_final = df_final.merge(df_other, on="time", how="outer")

    # Save
    filename = blob_name.split("/")[-1].replace(".tdms", ".parquet")
    output_path = f"{OUTPUT_PREFIX}{filename}"

    buffer = io.BytesIO()
    df_final.to_parquet(buffer, index=False)
    buffer.seek(0)

    bucket.blob(output_path).upload_from_file(buffer, content_type="application/octet-stream")

    print(f"✅ Saved to: {output_path}")


def main():
    print("📁 Listing files...")
    files = list_files(INPUT_PREFIX)
    print(f"🔎 Found {len(files)} TDMS files.\n")

    for f in tqdm(files):
        convert_tdms_to_parquet(f)

    print("\n🎉 Conversion completed. Files stored in current_temp_short/.")


if __name__ == "__main__":
    main()


📁 Listing files...
🔎 Found 45 TDMS files.



  0%|          | 0/45 [00:00<?, ?it/s]


🔄 Processing: current,temp/0Nm_BPFI_03.tdms


  2%|▏         | 1/45 [00:13<09:51, 13.44s/it]

✅ Saved to: current_temp_short/0Nm_BPFI_03.parquet

🔄 Processing: current,temp/0Nm_BPFI_10.tdms


...

✅ Saved to: current_temp_short/0Nm_Misalign_01.parquet

🔄 Processing: current,temp/0Nm_Misalign_03.tdms


 18%|█▊        | 8/45 [01:42<09:40, 15.70s/it]

✅ Saved to: current_temp_short/0Nm_Misalign_03.parquet

🔄 Processing: current,temp/0Nm_Misalign_05.tdms


 20%|██        | 9/45 [01:51<08:06, 13.52s/it]

✅ Saved to: current_temp_short/0Nm_Misalign_05.parquet

🔄 Processing: current,temp/0Nm_Normal.tdms
